In [20]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image

In [40]:
IMG_SIZE = (40, 40)
IMG_SHAPE = (40, 40, 3)
NOF_CLASSES = 43

TRAIN_SET_PATH = '/home/viktor/workspace/00_datasets/GTSRB_40x40/training_set_40x40'
TRAIN_SET_SIZE = 39209
BATCH_SIZE = 128
TRAIN_STEPS = TRAIN_SET_SIZE // BATCH_SIZE

VALID_SET_PATH = '/home/viktor/workspace/00_datasets/GTSRB_40x40/test_set_40x40'
VALID_SET_SIZE = 12630
VALID_BATCH_SIZE = 64
VALID_STEPS = VALID_SET_SIZE // VALID_BATCH_SIZE

In [44]:
# Flow training images in batches of 128 using train_datagen generator
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        TRAIN_SET_PATH,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='sparse')

valid_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        VALID_SET_PATH,
        target_size=IMG_SIZE,
        batch_size=VALID_BATCH_SIZE,
        class_mode='sparse')

Found 39209 images belonging to 43 classes.
Found 12630 images belonging to 43 classes.


In [46]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
#     # This is the first convolution
#     tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=IMG_SHAPE),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     # The second convolution
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     # The third convolution
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(input_shape=IMG_SHAPE),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(NOF_CLASSES, activation='softmax')
])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4800)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               2458112   
_________________________________________________________________
dense_5 (Dense)              (None, 43)                22059     
Total params: 2,480,171
Trainable params: 2,480,171
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [48]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=TRAIN_STEPS,  
      epochs=15,
      verbose=2,
      validation_data = valid_generator,
      validation_steps=VALID_STEPS)

Epoch 1/15
 - 6s - loss: 2.9544 - acc: 0.3286 - val_loss: 1.9789 - val_acc: 0.4817
Epoch 2/15
 - 6s - loss: 1.4296 - acc: 0.6252 - val_loss: 1.4569 - val_acc: 0.6350
Epoch 3/15
 - 6s - loss: 0.9905 - acc: 0.7377 - val_loss: 1.4410 - val_acc: 0.6507
Epoch 4/15
 - 6s - loss: 0.7708 - acc: 0.7984 - val_loss: 1.0756 - val_acc: 0.7157
Epoch 5/15
 - 6s - loss: 0.6487 - acc: 0.8296 - val_loss: 1.8867 - val_acc: 0.5987
Epoch 6/15
 - 6s - loss: 0.5584 - acc: 0.8552 - val_loss: 0.9187 - val_acc: 0.7877
Epoch 7/15
 - 6s - loss: 0.4919 - acc: 0.8701 - val_loss: 1.0188 - val_acc: 0.7688
Epoch 8/15
 - 6s - loss: 0.4469 - acc: 0.8816 - val_loss: 0.8143 - val_acc: 0.8160
Epoch 9/15
 - 6s - loss: 0.4054 - acc: 0.8947 - val_loss: 0.8628 - val_acc: 0.8108
Epoch 10/15
 - 6s - loss: 0.3781 - acc: 0.9025 - val_loss: 1.4429 - val_acc: 0.7178
Epoch 11/15
 - 6s - loss: 0.3615 - acc: 0.9077 - val_loss: 0.8750 - val_acc: 0.8233
Epoch 12/15
 - 6s - loss: 0.3277 - acc: 0.9136 - val_loss: 0.8977 - val_acc: 0.8109
E

In [49]:
img = image.load_img(TEST_IMG_PATH, target_size=IMG_SIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(np.where(classes[0])[0])

FileNotFoundError: [Errno 2] No such file or directory: '/home/viktor/workspace/00_datasets/GTSRB_40x40/test_set_40x40/00035.ppm'

In [35]:
from shutil import move
df = pd.read_csv('/home/viktor/workspace/00_datasets/GTSRB_40x40/test_set_40x40/GT-final_test.csv', delimiter=';')
for index, row in df.iterrows():
    src_dir = '/home/viktor/workspace/00_datasets/GTSRB_40x40/test_set_40x40'
    src_file = src_dir + '/' + row['Filename']
    dst_dir = '/home/viktor/workspace/00_datasets/GTSRB_40x40/test_set_40x40/' + f'{row["ClassId"]:05d}'
    dst_file = dst_dir + '/' + row['Filename']
    
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    move(src_file, dst_file)